In [2]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


candlestick_frame = 60*4
pnl_threshold = 8
prev_sma_count = 10


df = pd.read_csv('../../01-11-2019  15-08-2023.csv')
df['sma_7'] = ta.sma(df['close'], length=7)

ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
sma_7 = list(df['sma_7'])


tot = len(ts)
sma_7_to_close_pct_lst = []
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    sma_7_to_close_pct_calc = sma_7[idx]*100/close[idx-1]
    sma_7_to_close_pct_lst.append(round(sma_7_to_close_pct_calc-100, 6)) if sma_7_to_close_pct_calc != None else sma_7_to_close_pct_lst.append(0)
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(high[idx:idx+candlestick_frame])
        min_low = min(low[idx:idx+candlestick_frame])
        entry_price = open[idx]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     

sma_7_to_close_pct = sma_7_to_close_pct_lst
long_runup = long_runup_lst
long_drawdown = long_drawdown_lst
short_runup = short_runup_lst
short_drawdown = short_drawdown_lst
processed_df = df


long=[]
short=[]
dont_trade=[]

tot = len(ts)

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        app = {}
        for i in range(prev_sma_count, -1, -1):
            app[f'sma_7_to_close_pct_{i}'] = sma_7_to_close_pct[idx-i]
        app['sma_7_to_close_pct_sum'] = sum(sma_7_to_close_pct[idx-prev_sma_count: idx])
        if long_runup[idx] >= pnl_threshold:
            app['signal']='long'
            long.append(app)
        elif short_runup[idx] >= pnl_threshold:
            app['signal']='short'
            short.append(app)
        else:
            app['signal']='dont_trade'
            dont_trade.append(app)

dataset = []
for idx in range(len(long)):
    dataset.append(long[idx])
    dataset.append(short[idx])
    dataset.append(dont_trade[idx])

print('saving dataset.csv....')
dataset_fn = f'../../dataset.csv'
pd.DataFrame(dataset).to_csv(dataset_fn, index=False)




saving dataset.csv....
